<h1>Design Patterns for ML projects</h1>

<h2>1.Reusable Training loop ('train' method in 'trainer.py')</h2>

In [ ]:
def train_loop(model, optimizer, loss_fn, dataloader, epochs):  
    for epoch in range(epochs):                     # epoch loop
        for inputs, targets in dataloader:          # batch loop
            optimizer.zero_grad()                     # clears the gradients from previous batch
            outputs = model(inputs)                   # forward pass
            loss = loss_fn(outputs, targets)          # compute loss
            loss.backward()                           # back propagation
            optimizer.step()                          # update params
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


train_loop(model, optimizer, loss_fn, train_loader, epochs=10)

<h2>2.Design Patterns</h2>

<h3>(1) Decorator (train.py)</h3>
    
<h7> Decorator adds logging functionality without modifying the core training code, keeping the code modular and clean.
In this example this is done by simplely adding @log_training above the train_loop function. </h7>

In [ ]:
import time

def log_training(func):
    def wrapper(*args, **kwargs):       # **kwargs (keyword arguments captures the key-value pairs)
        start_time = time.time()
        print(f"Starting training: {func.__name__}...")
        
        result = func(*args, **kwargs)  # Call actual training function
        
        end_time = time.time()
        print(f"Completed training: {func.__name__}. Time taken: {end_time - start_time:.2f}s")
        return result
    return wrapper

@log_training                          # this wraps the original function 'train_loop' with 'log_training'
def train_loop(model, optimizer, loss_fn, dataloader, epochs):
    for epoch in range(epochs):
        epoch_loss = 0
        for inputs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")

# Assuming model, optimizer, loss_fn, and dataloader are predefined
train_loop(model, optimizer, loss_fn, dataloader, epochs=5)

<h3>(3) Facade pattern (trainer.py)</h3>
    
<h7> Facade builds end-to-end pipelines for ML models, hiding the intricacy of each component (data loading, feature engineering, model training, evaluation) 
     </h7>

In [ ]:
class DataLoader:                             # Load data 
    def load_data(self):
        print("Loading data")

class Preprocessor:                           # Process data 
    def preprocess_data(self, data):
        print("Preprocessing data")

class ModelTrainer:                           # Train model 
    def train_model(self, processed_data):
        print("Training model")


class ModelPipelineFacade:                    # pipeline
    def __init__(self):
        self.data_loader = DataLoader()
        self.preprocessor = Preprocessor()
        self.trainer = ModelTrainer()

    def run_pipeline(self):
        data = self.data_loader.load_data()
        processed_data = self.preprocessor.preprocess_data(data)
        model = self.trainer.train_model(processed_data)
        print("Pipeline complete")

# Usage
pipeline = ModelPipelineFacade()
pipeline.run_pipeline()

<h3>(4) Builder pattern (model.py)</h3>
    
<h7> Constructs a complex object (like a neural network architecture) step-by-step, allowing for flexible and customizable configurations.
     </h7>

In [ ]:

import torch
import torch.nn as nn

class ModelBuilder(nn.Module):
    def __init__(self):
        super(ModelBuilder, self).__init__()
        self.layers = nn.ModuleList()

    def add_layer(self, units, activation=None):
        if not hasattr(self, 'input_shape'):
            raise ValueError("You need to set input shape first with `set_input_shape`")

        # fully connected layer
        self.layers.append(nn.Linear(self.input_shape, units))

        # activation function
        if activation:
            self.layers.append(getattr(nn, activation)())

        # input shape for the next layer
        self.input_shape = units
        return self

    def set_input_shape(self, input_shape):
        self.input_shape = input_shape
        return self

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Usage
model = (ModelBuilder()
         .set_input_shape(28 * 28)  # Setting input shape for flattened 28x28 images
         .add_layer(128, 'ReLU')     # Adding hidden layer
         .add_layer(10, 'Softmax'))  # Adding output layer with softmax

In [ ]:
<h3>(5) Observer pattern </h3>
    